CHECK VERSION GPU KAGGLE

# Medical Machine Translation - VLSP 2025

## 1. Kiểm tra môi trường và cấu hình GPU

Trước khi bắt đầu, chúng ta kiểm tra cấu hình phần cứng (GPU Tesla T4) và các thư viện deep learning (PyTorch, CUDA) để đảm bảo môi trường phù hợp cho việc huấn luyện mô hình ngôn ngữ lớn.

In [ ]:
import torch
import subprocess

print("===== GPU =====")
print(subprocess.getoutput("nvidia-smi"))

print("\n===== CUDA Torch =====")
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Torch CUDA version:", torch.version.cuda)

## 2. Cài đặt thư viện

### 2.1. Gỡ các phiên bản cũ
Để tránh xung đột phiên bản, chúng ta gỡ bỏ các thư viện cũ trước khi cài đặt phiên bản tương thích với Qwen2.5-3B.

In [ ]:
!pip uninstall -q -y transformers accelerate peft trl bitsandbytes unsloth unsloth-zoo protobuf

### 2.2. Cài đặt Dependencies chính

Cài đặt các thư viện cốt lõi:
- **transformers**: Thư viện Hugging Face để load mô hình Qwen2.5
- **peft**: Parameter-Efficient Fine-Tuning (LoRA)
- **trl**: Trainer cho Supervised Fine-Tuning
- **sacrebleu**: Đánh giá chất lượng dịch thuật
- **sentence-transformers**: Xử lý embedding

In [ ]:
!pip install -q --no-cache-dir \
    "protobuf==3.20.3" \
    "transformers==4.57.3" \
    "accelerate==1.12.0" \
    "peft==0.13.2" \
    "trl==0.24.0" \
    "datasets>=2.13.0" \
    "safetensors" \
    "sentencepiece" \
    "huggingface-hub" \
    "evaluate" \
    "sacrebleu" \
    "sentence-transformers"

print("Done")

## 3. Load Mô hình và Tokenizer

### Mô hình cơ sở: Qwen2.5-3B-Instruct
Mô hình cơ sở có 3 tỷ tham số, được huấn luyện sẵn với khả năng:
- Đa ngôn ngữ (bao gồm tiếng Việt)
- Tuân thủ chỉ dẫn (Instruction Following)
- Cửa sổ ngữ cảnh 32K tokens

**Lưu ý**: Nhóm **KHÔNG** sử dụng quantization (4-bit/8-bit) để bảo toàn năng lực hiểu ngữ nghĩa y khoa.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    dtype="auto",              
    trust_remote_code=True,
    quantization_config=None,
)

print("Done")

## 4. Chuẩn bị và phân tích dữ liệu

### 4.1. Load dữ liệu Thô
Tập dữ liệu VLSP Medical bao gồm:
- **Train**: 500,000 cặp câu song ngữ Anh-Việt
- **Public Test**: 3,000 cặp câu

Dữ liệu chứa các thuật ngữ y khoa phức tạp như tên thuốc, bệnh lý, chỉ định lâm sàng và từ viết tắt chuyên ngành.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

DATA_DIR = "/kaggle/input/medical-data"
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

def read_file(path):
    with open(path, "r", encoding="utf-8") as f:
        return [line.strip() for line in f]

print("Loading raw text files...")
train_en = read_file(os.path.join(DATA_DIR, "train.en.txt"))
train_vi = read_file(os.path.join(DATA_DIR, "train.vi.txt"))
test_en = read_file(os.path.join(DATA_DIR, "public_test.en.txt"))
test_vi = read_file(os.path.join(DATA_DIR, "public_test.vi.txt"))

assert len(train_en) == len(train_vi)
print(f"Successfully loaded {len(train_en)} training pairs and {len(test_en)} test pairs.")

### 4.2. Phân tích thống kê và nhận diện nhiễu

#### Mục tiêu:
- Phân tích phân bổ độ dài câu
- Nhận diện các từ viết tắt y khoa (Medical Abbreviations)
- Phát hiện dữ liệu ngoại lai (outliers) do lỗi phân đoạn văn bản

#### Kết quả quan sát:
- Độ dài trung bình: ~21 từ (EN), ~30 từ (VI)
- Độ dài cực đại: 477 từ (EN), 519 từ (VI) → **Dữ liệu nhiễu**
- Hàng ngàn thuật ngữ viết tắt: CT, MRI, HIV, ĐTĐ, BN...

In [ ]:
ROMAN_NUMERALS = {'II', 'III', 'IV', 'VI', 'VII', 'VIII', 'IX', 'X', 'XI', 'XII'}

def analyze_stats_pro(data_list):
    lengths = [len(sentence.split()) for sentence in data_list]
    abbs = []
    for sentence in data_list:
        clean_sentence = re.sub(r'[^\w\s]', '', sentence)
        words = clean_sentence.split()
        for w in words:
            if w.isupper() and len(w) >= 2 and not w.isdigit():
                if w not in ROMAN_NUMERALS:
                    abbs.append(w)
    return lengths, abbs

# --- Execution ---
print("Analyzing Original Dataset Statistics...")
en_lengths_orig, en_abbs_orig = analyze_stats_pro(train_en)
vi_lengths_orig, vi_abbs_orig = analyze_stats_pro(train_vi)

# Stats Table
summary_orig = pd.DataFrame({
    "Metrics": ["Total Sentences", "Average Length (words)", "Max Length (words)", "Unique Abbreviations"],
    "English Corpus": [len(train_en), f"{np.mean(en_lengths_orig):.2f}", np.max(en_lengths_orig), len(set(en_abbs_orig))],
    "Vietnamese Corpus": [len(train_vi), f"{np.mean(vi_lengths_orig):.2f}", np.max(vi_lengths_orig), len(set(vi_abbs_orig))]
})


print("\n" + "="*60)
print("STATISTICAL SUMMARY OF ORIGINAL DATASET")
print("="*60)
print(summary_orig.to_string(index=False))
print("="*60)

# Chart 1: Sentence Length Distribution (Original)
plt.figure(figsize=(12, 6))
sns.histplot(vi_lengths_orig, color="#3498db", label="Vietnamese", kde=True, binrange=(0, 140), alpha=0.6)
sns.histplot(en_lengths_orig, color="#e74c3c", label="English", kde=True, binrange=(0, 100), alpha=0.4)
plt.title("Original Sentence Length Distribution", fontsize=16, fontweight='bold')
plt.xlabel("Number of Words")
plt.ylabel("Count")
plt.xlim(0, 150)
plt.legend()
plt.tight_layout()
plt.show()

# Helper Function for Abbreviation Charts
def plot_medical_abbs(abbs, title, palette, filename):
    top_abbs = Counter(abbs).most_common(15)
    words, counts = zip(*top_abbs)
    
    plt.figure(figsize=(12, 5))
    ax = sns.barplot(x=list(words), y=list(counts), palette=palette, hue=list(words), legend=False)
    
    # Add exact counts on top of bars
    for p in ax.patches:
        ax.annotate(f'{int(p.get_height())}', 
                    (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha = 'center', va = 'center', 
                    xytext = (0, 9), 
                    textcoords = 'offset points',
                    fontsize=10)

    plt.title(title, fontsize=15, fontweight='bold')
    plt.xlabel("Abbreviations", fontsize=12)
    plt.ylabel("Count", fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(filename, dpi=300)
    plt.show()

# Chart 2 & 3: Original Medical Abbreviations
print("Generating Original Abbreviation Charts...")
plot_medical_abbs(vi_abbs_orig, "Top 15 Medical Abbreviations (Original VI Dataset)", "viridis", "orig_vi_abbs.png")
plot_medical_abbs(en_abbs_orig, "Top 15 Medical Abbreviations (Original EN Dataset)", "magma", "orig_en_abbs.png")

# Outlier Identification (Noise detection)
max_vi_idx = np.argmax(vi_lengths_orig)
max_en_idx = np.argmax(en_lengths_orig)
print("\n" + "-"*60)
print(f"DEBUG: Longest VI Sample ({vi_lengths_orig[max_vi_idx]} words):")
print(f"{train_vi[max_vi_idx][:300]}...") 
print(f"\nDEBUG: Longest EN Sample ({en_lengths_orig[max_en_idx]} words):")
print(f"{train_en[max_en_idx][:300]}...")
print("-"*60)

### 4.3. Làm sạch dữ liệu

#### Chiến lược lọc:
Áp dụng ngưỡng **180 từ tổng** (EN + VI) cho mỗi cặp câu để:
- Đảm bảo toàn bộ prompt + dịch < 512 tokens (giới hạn của Qwen2.5)
- Tránh cắt cụt (truncation) gây mất thông tin y khoa quan trọng
- Loại bỏ các đoạn văn bị dính chùm do lỗi segmentation

#### Kết quả:
- **Loại bỏ**: 4,360 cặp câu nhiễu (~0.87%)
- **Giữ lại**: 495,640 cặp câu chất lượng cao
- Độ dài cực đại giảm xuống: 126 từ (EN), 136 từ (VI)

In [ ]:
train_pairs = [{"en": e, "vi": v} for e, v in zip(train_en, train_vi)]
test_pairs = [{"en": e, "vi": v} for e, v in zip(test_en, test_vi)]

# --- Làm sạch (Ngưỡng: tổng cộng 180 từ) ---
# Điều này đảm bảo rằng (Câu hỏi + Nguồn + Mục tiêu) < 512 từ
MAX_WORDS_TOTAL = 180 

cleaned_pairs = [d for d in train_pairs if (len(d['en'].split()) + len(d['vi'].split())) < MAX_WORDS_TOTAL]

print(f"Original pairs count: {len(train_pairs)}")
print(f"Cleaned pairs count:  {len(cleaned_pairs)}")
print(f"Pruned {len(train_pairs) - len(cleaned_pairs)} noisy/oversized sequences.")

# --- Split After Cleaning ---
train_split, valid_split = train_test_split(
    cleaned_pairs, 
    test_size=0.05, 
    random_state=42, 
    shuffle=True
)

# --- Create Final DatasetDict ---
raw_dataset = DatasetDict({
    "train": Dataset.from_list(train_split),
    "validation": Dataset.from_list(valid_split),
    "test": Dataset.from_list(test_pairs),
})

print(f"\nDataset Construction Completed:")
print(f"- Final Train size: {len(train_split)}")
print(f"- Final Valid size: {len(valid_split)}")
print(f"- Final Test size:  {len(test_pairs)}")

### 4.4. Phân tích sau làm sạch

So sánh phân bổ dữ liệu và mật độ thuật ngữ y khoa **trước và sau** khi làm sạch:
- Phân bổ độ dài câu trở nên tập trung hơn (10-40 từ)
- Các thuật ngữ cốt lõi được bảo toàn hoàn toàn
- Loại bỏ hoàn toàn hiện tượng "đuôi dài" (long tail) gây nhiễu huấn luyện

In [ ]:
def plot_medical_abbs(abbs, title, palette, filename):
    top_abbs = Counter(abbs).most_common(15)
    words, counts = zip(*top_abbs)
    plt.figure(figsize=(12, 5))
    ax = sns.barplot(x=list(words), y=list(counts), palette=palette, hue=list(words), legend=False)
    for p in ax.patches:
        ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha='center', va='center', xytext=(0, 9), textcoords='offset points', fontsize=10)
    plt.title(title, fontsize=15, fontweight='bold')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(filename, dpi=300)
    plt.show()

clean_train_en = [d['en'] for d in cleaned_pairs]
clean_train_vi = [d['vi'] for d in cleaned_pairs]

en_lengths_clean, en_abbs_clean = analyze_stats_pro(clean_train_en)
vi_lengths_clean, vi_abbs_clean = analyze_stats_pro(clean_train_vi)

# Chart Cleaned Distribution
plt.figure(figsize=(12, 6))
sns.histplot(vi_lengths_clean, color="#2ecc71", label="Vietnamese (Cleaned)", kde=True, alpha=0.6)
sns.histplot(en_lengths_clean, color="#f39c12", label="English (Cleaned)", kde=True, alpha=0.4)
plt.title("Cleaned Sentence Length Distribution (Safety Limit Applied)", fontsize=16, fontweight='bold')
plt.xlabel("Number of Words")
plt.legend()
plt.show()

# Charts Abbreviation analysis for Cleaned Data
plot_medical_abbs(en_abbs_clean, "Top 15 Medical Abbreviations (Cleaned EN Dataset)", "magma", "clean_en_abbs.png")
plot_medical_abbs(vi_abbs_clean, "Top 15 Medical Abbreviations (Cleaned VI Dataset)", "viridis", "clean_vi_abbs.png")

# Final Comparison Table
summary_clean = pd.DataFrame({
    "Metrics": ["Total Sentences", "Average Length (words)", "Max Length (words)", "Unique Abbreviations"],
    "English (Cleaned)": [len(clean_train_en), f"{np.mean(en_lengths_clean):.2f}", np.max(en_lengths_clean), len(set(en_abbs_clean))],
    "Vietnamese (Cleaned)": [len(clean_train_vi), f"{np.mean(vi_lengths_clean):.2f}", np.max(vi_lengths_clean), len(set(vi_abbs_clean))]
})

print("\n" + "="*60)
print("STATISTICAL SUMMARY OF CLEANED DATASET")
print("="*60)
print(summary_clean.to_string(index=False))
print("="*60)

## 5. Cấu hình hướng huấn luyện

### Chiến lược đa chiều linh hoạt

Thay vì huấn luyện một mô hình "vạn năng" cho cả hai chiều (dễ bị Task Interference), chúng ta huấn luyện **hai Adapter độc lập**:

- **`TRAIN_DIRECTION = "en_vi"`**: Huấn luyện adapter chuyên dịch Anh → Việt
- **`TRAIN_DIRECTION = "vi_en"`**: Huấn luyện adapter chuyên dịch Việt → Anh

Mỗi adapter học sâu vào đặc trưng ngôn ngữ và thuật ngữ của một chiều duy nhất.

In [ ]:
# Chọn hướng: "en_vi" (Anh->Việt) hoặc "vi_en" (Việt->Anh)
#TRAIN_DIRECTION = "vi_en"
TRAIN_DIRECTION = "en_vi"
print(f"Chế độ Train: {TRAIN_DIRECTION.upper()}")

### 5.1. Xây dựng Prompt Template

#### Chuyển đổi dịch máy → hội thoại dẫn dắt

Qwen2.5 là mô hình Decoder-only (không phải Encoder-Decoder như BART), do đó chúng ta sử dụng định dạng **ChatML** để biến bài toán dịch thuật thành bài toán Instruction Following:

**Ví dụ với EN-VI:**
```
User: Dịch câu sau sang tiếng Việt:
      <Câu tiếng Anh>
Assistant: <Câu tiếng Việt>
```

Hàm `build_prompt()` tự động xây dựng prompt phù hợp dựa trên `TRAIN_DIRECTION` hiện tại.

In [ ]:
def build_prompt(src_text, tgt_text=None):
    """
    Tạo prompt theo chuẩn chat của Qwen2.5:
    - user: yêu cầu dịch dựa trên TRAIN_DIRECTION
    - assistant: ground truth (tgt_text) nếu training
    """
    # Xác định hướng và tạo prompt tương ứng
    if TRAIN_DIRECTION == "en_vi":
        user_content = f"Dịch câu sau sang tiếng Việt:\n{src_text}"
    else: # vi_en
        user_content = f"Dịch câu sau sang tiếng Anh:\n{src_text}"

    messages = [{"role": "user", "content": user_content}]
    
    if tgt_text is not None:
        messages.append({"role": "assistant", "content": tgt_text})

    return tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False
    )

## 6. Đăng nhập Hugging Face Hub

Cần thiết để:
- Lưu trữ dữ liệu đã tokenize (Smart Caching)
- Đẩy adapter lên cloud sau khi huấn luyện (Safe-Save Protocol)
- Đảm bảo tính liên tục khi Kaggle Kernel khởi động lại
- Lưu file được model dịch sau khi train

In [ ]:
from huggingface_hub import login

HF_TOKEN = "token_login" 

login(token=HF_TOKEN)
print(">>> Login success!")

### 6.1. Tokenization và Smart Caching

#### Quy trình:
1. **Check**: Kiểm tra xem dữ liệu đã tokenize cho `TRAIN_DIRECTION` hiện tại có trên Hub chưa
2. **Reuse**: Nếu có → Tải về và sử dụng trực tiếp (tiết kiệm quá trình tokenize lại)
3. **Update**: Nếu chưa có → Tokenize toàn bộ dataset và đẩy lên Hub

#### Kết quả Tokenization:
- **Train**: 470,858 mẫu
- **Validation**: 24,782 mẫu  
- **Test**: 3,000 mẫu

Mỗi mẫu được mã hóa với `max_length=512`, bao gồm:
- `input_ids`: Chuỗi token của prompt + câu nguồn + câu đích
- `attention_mask`: Mask phân biệt token thực và padding
- `labels`: Copy của `input_ids` để tính loss (Causal Language Modeling)

In [ ]:
from datasets import load_dataset

HF_USERNAME = "yuiyL"
# Tên Repo Data sẽ đổi theo hướng: ...-tokenized-en-vi HOẶC ...-tokenized-vi-en
TOKENIZED_DATASET_REPO = f"{HF_USERNAME}/qwen2.5-medical-tokenized-{TRAIN_DIRECTION}"
def preprocess(example):
    # Đảo chiều input dựa trên cấu hình
    if TRAIN_DIRECTION == "en_vi":
        src, tgt = example["en"], example["vi"]
    else: # vi_en
        src, tgt = example["vi"], example["en"]

    prompt = build_prompt(src, tgt)
    tokenized = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding=False
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized


print(f">>> Checking tokenized dataset: {TOKENIZED_DATASET_REPO}")

try:
    tokenized_dataset = load_dataset(TOKENIZED_DATASET_REPO)
    print("Loaded tokenized dataset from Hub")

except Exception as e:
    print(f"Tokenized dataset not found for {TRAIN_DIRECTION}. Running tokenization...")

    tokenized_dataset = raw_dataset.map(
        preprocess,
        remove_columns=["en", "vi"],
        desc=f"Tokenizing medical dataset ({TRAIN_DIRECTION})"
    )

    print(">>> Pushing tokenized dataset to Hub")
    tokenized_dataset.push_to_hub(
        TOKENIZED_DATASET_REPO,
        private=True
    )

    print("Tokenized dataset uploaded successfully")

## 7. Cấu hình LoRA và khởi tạo Training

### 7.1. LoRA Configuration

**Low-Rank Adaptation (LoRA)** là kỹ thuật PEFT giúp fine-tune mô hình 3B trên GPU 16GB:

- **Rank (r=16)**: Kích thước ma trận thích ứng
- **Alpha (α=32)**: Hệ số khuếch đại tín hiệu học
- **Dropout (0.05)**: Chống overfitting
- **Target Modules**: Áp dụng LoRA vào toàn bộ lớp tuyến tính (q, k, v, o, gate, up, down)

### 7.2. Safe-Save Protocol

Hệ thống kiểm tra Hub để:
- **Resume**: Nếu đã có adapter → Tải về và tiếp tục huấn luyện
- **Fresh Start**: Nếu chưa có → Khởi tạo LoRA mới

File `best_state.json` lưu giá trị `best_eval_loss` để so sánh hiệu năng qua các phiên.

In [ ]:
import os
import json
import torch
from transformers import TrainingArguments, DataCollatorForSeq2Seq, EarlyStoppingCallback
from peft import LoraConfig, TaskType, PeftModel
from trl import SFTTrainer
from huggingface_hub import HfApi, hf_hub_download

import gc
def reset_cuda():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    gc.collect()


HF_USERNAME = "yuiyL"
# Tên Model cũng đổi theo hướng: ...-sft-en-vi HOẶC ...-sft-vi-en
REPO_NAME = f"qwen2.5-3b-medical-sft-{TRAIN_DIRECTION}"
HUB_MODEL_ID = f"{HF_USERNAME}/{REPO_NAME}"

OUTPUT_DIR = f"qwen2.5-3b-medical-sft-{TRAIN_DIRECTION}"
FINAL_ADAPTER_DIR = f"{OUTPUT_DIR}/final_adapter"
STATE_FILE_NAME = "best_state.json"
LOCAL_STATE_FILE = f"{FINAL_ADAPTER_DIR}/{STATE_FILE_NAME}"
os.makedirs(FINAL_ADAPTER_DIR, exist_ok=True)

api = HfApi()


best_prev_loss = float("inf")
print(f">>> Checking Hub history: {HUB_MODEL_ID}")

try:
    downloaded_path = hf_hub_download(
        repo_id=HUB_MODEL_ID,
        filename=STATE_FILE_NAME,
        local_dir=FINAL_ADAPTER_DIR
    )
    with open(downloaded_path, "r") as f:
        best_prev_loss = json.load(f).get("best_eval_loss", float("inf"))
    print(f"Found previous best loss: {best_prev_loss:.4f}")
except Exception:
    print("No previous record found. Fresh run.")


peft_config = None
has_adapter_on_hub = False

try:
    if "adapter_model.safetensors" in api.list_repo_files(HUB_MODEL_ID):
        has_adapter_on_hub = True
except:
    pass

if has_adapter_on_hub:
    print(">>> RESUME: Loading adapter from Hub")
    model = PeftModel.from_pretrained(model, HUB_MODEL_ID, is_trainable=True)
else:
    print(">>> FRESH START: Initializing new LoRA")
    peft_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
        target_modules=[
            "q_proj", "k_proj", "v_proj",
            "o_proj", "gate_proj", "up_proj", "down_proj"
        ]
    )

### 7.3. Training Arguments
Cell này chứa cấu hình siêu tham số huấn luyện:

**Chiến lược đa giai đoạn:**
- **Giai đoạn 1 (Khởi đầu)**: 10K mẫu, 800 steps, LR=2e-4
- **Giai đoạn 2 (Tinh chỉnh)**: 20K mẫu, 1600 steps, LR=5e-5  
- **Giai đoạn 3 (Hội tụ sâu)**: 20K mẫu, 1600 steps, LR=1e-5

**Tối ưu hóa VRAM:**
- Batch size = 4 (per device)
- Gradient accumulation = 4 → Effective batch = 16
- Mixed Precision (BF16)

**Giám sát:**
- Eval every 200 steps
- Early Stopping patience = 2
- Load best model at end

In [ ]:

VALID_SIZE = 500
TRAIN_SIZE = 20000
MAX_STEPS  = 1600
EVAL_STEPS = 200

train_subset = tokenized_dataset["train"].shuffle(seed=42).select(range(TRAIN_SIZE))
valid_subset = tokenized_dataset["validation"].select(range(VALID_SIZE))

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    pad_to_multiple_of=8,
    padding=True,
    return_tensors="pt"
)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,

    max_steps=MAX_STEPS,               
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    lr_scheduler_type = "cosine",
    warmup_steps=120,   

    per_device_eval_batch_size=2,

    eval_strategy="steps",         
    eval_steps=EVAL_STEPS,             
    save_strategy="steps",             
    save_steps=EVAL_STEPS,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",

    bf16=torch.cuda.is_bf16_supported(),
    fp16=not torch.cuda.is_bf16_supported(),

    logging_steps=50,
    report_to="none",
    remove_unused_columns=False,
    push_to_hub=False
)


### 7.4. Training Execution

Cell này thực thi quá trình huấn luyện và áp dụng **Best-Model-Only Policy**:

1. **Train**: SFTTrainer huấn luyện với Early Stopping
2. **Compare**: So sánh `current_run_best_loss` với `best_prev_loss`
3. **Save & Push**: Chỉ cập nhật Hub nếu model mới tốt hơn

**Lợi ích:**
- Tiết kiệm băng thông (chỉ push checkpoint tốt nhất)
- Đảm bảo tính liên tục qua các phiên Kaggle
- Tận dụng tối đa 500K mẫu qua nhiều phiên huấn luyện (nhờ shuffle seed=42)

In [ ]:

trainer = SFTTrainer(
    model=model,
    train_dataset=train_subset,
    eval_dataset=valid_subset,
    args=training_args,
    peft_config=peft_config,
    data_collator=data_collator,
    callbacks=[
        # SafeBleuEvalCallback(tokenizer, test_pairs, num_samples=50),
        EarlyStoppingCallback(early_stopping_patience=2),
    ],
)

print("\nReset CUDA before training")
reset_cuda()

print("\n>>> START TRAINING")
trainer.train()



current_run_best_loss = trainer.state.best_metric
if current_run_best_loss is None:
    current_run_best_loss = trainer.evaluate()["eval_loss"]

print("\n" + "=" * 50)
print(f"Previous best loss : {best_prev_loss:.4f}")
print(f"Current run loss   : {current_run_best_loss:.4f}")

if current_run_best_loss < best_prev_loss:
    print("New model is better. Saving & pushing to Hub.")

    trainer.save_model(FINAL_ADAPTER_DIR)
    tokenizer.save_pretrained(FINAL_ADAPTER_DIR)

    with open(LOCAL_STATE_FILE, "w") as f:
        json.dump({"best_eval_loss": current_run_best_loss}, f)

    api.upload_folder(
        folder_path=FINAL_ADAPTER_DIR,
        repo_id=HUB_MODEL_ID,
        repo_type="model",
        commit_message=f"Upgrade: loss {current_run_best_loss:.4f}"
    )

    print(f"Updated model: https://huggingface.co/{HUB_MODEL_ID}")
else:
    print("Model not improved. Skip pushing.")

print("=" * 50)


## 8. Phân tích Learning Curves
Do quá trình train không cần thực thi lại nên sẽ viết code để vẽ theo dữ liệu đã được train (ảnh)
### Trực quan hóa quá trình học

Cell này vẽ đường cong **Training Loss** và **Validation Loss** qua các steps để:

#### Nhận diện hội tụ tối ưu:
- **EN-VI**: Best model tại Step 1200 (Val Loss = 1.0936)
- **VI-EN**: Best model tại Step 1200 (Val Loss = 1.0726)

#### Nhận diện Overfitting:
- Training Loss giảm mạnh nhưng Validation Loss tăng
- Đánh dấu bằng dấu sao vàng (⭐) tại điểm tối ưu

**Insight**: Cả hai chiều đều đạt đỉnh hiệu năng tại cùng một checkpoint, chứng minh tính ổn định của cấu hình LoRA.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Thiết lập phong cách academic
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (14, 7)

def plot_training_metrics(steps, train_loss, val_loss, title, filename):
    fig, ax = plt.subplots()
    
    line1, = ax.plot(steps, train_loss, 'b-o', label='Training Loss', linewidth=2, markersize=7, alpha=0.7)
    line2, = ax.plot(steps, val_loss, 'r-s', label='Validation Loss', linewidth=2, markersize=7, alpha=0.7)
    
    for i, (s, tl, vl) in enumerate(zip(steps, train_loss, val_loss)):
        ax.text(s, tl + 0.005, f'{tl:.3f}', color='blue', ha='center', va='bottom', fontsize=9, fontweight='bold')
        ax.text(s, vl - 0.005, f'{vl:.3f}', color='red', ha='center', va='top', fontsize=9, fontweight='bold')
    
    min_val_loss = min(val_loss)
    idx_min = val_loss.index(min_val_loss)
    best_step = steps[idx_min]
    
    ax.scatter(best_step, min_val_loss, color='gold', s=350, marker='*', 
               edgecolors='black', zorder=10, label=f'Best Model')
    
    ax.annotate(f'BEST: {min_val_loss:.4f}', 
                xy=(best_step, min_val_loss),
                xytext=(0, -35), 
                textcoords='offset points',
                ha='center',
                fontsize=11, fontweight='bold',
                bbox=dict(boxstyle="round,pad=0.3", fc="yellow", ec="black", alpha=0.8))

    ax.set_title(title, fontsize=18, fontweight='bold', pad=20)
    ax.set_xlabel('Training Steps', fontsize=13)
    ax.set_ylabel('Loss (Cross Entropy)', fontsize=13)
    
    ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1), borderaxespad=0, frameon=True, shadow=True)
    
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout() 
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()

# EN-VI
steps_envi_good = [200, 400, 600, 800, 1000, 1200, 1400, 1600]
train_envi_good = [1.1528, 1.1285, 1.1191, 1.1227, 1.1070, 1.1212, 1.0142, 1.0165]
val_envi_good = [1.1326, 1.1247, 1.1143, 1.1059, 1.0993, 1.0936, 1.0971, 1.0972]

steps_envi_over = [200, 400, 600]
train_envi_over = [0.8120, 0.7886, 0.8186]
val_envi_over = [1.0902, 1.1021, 1.0920]

# VI-EN
steps_vien_good = [200, 400, 600, 800, 1000, 1200, 1400, 1600]
train_vien_good = [1.1258, 1.0915, 1.0821, 1.0857, 1.0700, 1.0842, 0.9772, 0.9795]
val_vien_good = [1.1116, 1.1037, 1.0933, 1.0849, 1.0783, 1.0726, 1.0762, 1.0763]

steps_vien_bad = [200, 400, 600]
train_vien_bad = [1.0170, 0.9919, 0.9928]
val_vien_bad = [1.0982, 1.1022, 1.1034]

plot_training_metrics(steps_envi_good, train_envi_good, val_envi_good, "EN-VI Translation: Optimal Convergence", "envi_good_v2.png")
plot_training_metrics(steps_envi_over, train_envi_over, val_envi_over, "EN-VI Translation: Overfitting Analysis", "envi_overfit_v2.png")
plot_training_metrics(steps_vien_good, train_vien_good, val_vien_good, "VI-EN Translation: Optimal Convergence", "vien_good_v2.png")
plot_training_metrics(steps_vien_bad, train_vien_bad, val_vien_bad, "VI-EN Translation: Non-improving Scenario", "vien_bad_v2.png")

## 9. Đánh giá định lượng (Sacre BLEU Score)

### 9.1. Cấu hình Đánh giá

- **UPDATE_ALL_TEST = True**: Dịch lại toàn bộ 3000 câu test bằng model hiện tại
- **UPDATE_ALL_TEST = False**: Chỉ load kết quả dịch đã lưu trên Hub

**Lưu trữ:** trên Hugging Face Hub
- `test_results_en_vi.parquet`: Kết quả dịch EN→VI
- `test_results_vi_en.parquet`: Kết quả dịch VI→EN

Mỗi file chứa:
- `source`: Câu nguồn
- `reference`: Ground truth
- `base_model`: Bản dịch của Base Model (zero-shot)
- `fine_tuned`: Bản dịch của Fine-tuned Model (LoRA)

In [ ]:
# --- CẤU HÌNH ĐÁNH GIÁ CHIẾN LƯỢC ---
UPDATE_ALL_TEST = True  # True: Dịcah lại 3000 câu bằng model hiện tại | False: Chỉ load file cũ
TEST_REPO_ID = f"{HF_USERNAME}/medical-test-results" # Tên repo chứa kết quả dịch

# Tên file cho từng chiều
FILE_EN_VI = "test_results_en_vi.parquet"
FILE_VI_EN = "test_results_vi_en.parquet"

print(f">>> Hướng huấn luyện chính: {TRAIN_DIRECTION.upper()}")
print(f">>> Chế độ cập nhật: {'BẮT ĐẦU DỊCH VÀ GHI ĐÈ' if UPDATE_ALL_TEST else 'CHỈ LOAD DỮ LIỆU ĐÃ LƯU'}")

### 9.2. Inference và Lưu trữ Kết quả

#### Quy trình:
1. **Check Hub**: Kiểm tra file kết quả có tồn tại chưa
2. **Translate**: Nếu `UPDATE_ALL_TEST=True`, dịch toàn bộ 3000 mẫu:
   - **Base Model**: Tắt adapter (`model.disable_adapter()`)
   - **Fine-tuned Model**: Bật adapter (mặc định)
3. **Save & Upload**: Lưu DataFrame → Parquet → Push lên Hub

**Lợi ích:**
- Tránh inference lặp lại (tiết kiệm ~30 phút GPU)
- Cho phép phân tích offline sau khi Kernel dừng
- Dễ dàng so sánh giữa các phiên bản model

In [ ]:
import pandas as pd
from huggingface_hub import hf_hub_download, upload_file
from tqdm import tqdm
import sacrebleu

def get_or_update_translations(model, tokenizer, dataset, direction, update=False):
    filename = FILE_EN_VI if direction == "en_vi" else FILE_VI_EN
    df_results = None
    
    try:
        path = hf_hub_download(repo_id=TEST_REPO_ID, filename=filename, repo_type="dataset")
        df_results = pd.read_parquet(path)
        print(f"Đã tải file kết quả cũ của {direction.upper()} ({len(df_results)} câu)")
    except:
        print(f"ℹChưa có dữ liệu cũ cho chiều {direction.upper()} trên Hub.")

    # Nếu update=True hoặc chưa có file -> Thực hiện dịch
    if update or df_results is None:
        print(f"Đang tiến hành dịch 3000 câu cho chiều {direction.upper()}...")
        src_key = "en" if direction == "en_vi" else "vi"
        tgt_key = "vi" if direction == "en_vi" else "en"
        prompt_tpl = "Dịch câu sau sang tiếng Việt:\n{}" if direction == "en_vi" else "Dịch câu sau sang tiếng Anh:\n{}"
        
        # Lấy toàn bộ 3000 mẫu
        samples = dataset.select(range(min(3000, len(dataset))))
        data = {"source": [], "reference": [], "base_model": [], "fine_tuned": []}

        model.eval()
        for item in tqdm(samples, desc=f"Translating {direction.upper()}"):
            src_text, ref_text = item[src_key], item[tgt_key]
            messages = [{"role": "user", "content": prompt_tpl.format(src_text)}]
            inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(model.device)

            with torch.no_grad():
                # Dịch bằng Base Model (Tắt adapter)
                with model.disable_adapter():
                    out_base = model.generate(inputs, max_new_tokens=512, do_sample=False)
                    data["base_model"].append(tokenizer.decode(out_base[0][len(inputs[0]):], skip_special_tokens=True).strip())
                
                # Dịch bằng Fine-tuned Model (Bật adapter)
                out_ft = model.generate(inputs, max_new_tokens=512, do_sample=False)
                data["fine_tuned"].append(tokenizer.decode(out_ft[0][len(inputs[0]):], skip_special_tokens=True).strip())
            
        df_results = pd.DataFrame(data)
        
        # Lưu và đẩy lên Hub ngay lập tức
        df_results.to_parquet(filename)
        upload_file(path_or_fileobj=filename, path_in_repo=filename, repo_id=TEST_REPO_ID, repo_type="dataset")
        print(f"Đã cập nhật và upload kết quả mới cho {direction.upper()}.")

    return df_results

# Thực thi cập nhật cho hướng hiện tại
test_data = raw_dataset["test"]
# Hàm này sẽ chỉ dịch nếu UPDATE_ALL_TEST = True, nếu không nó chỉ load df_results
current_df = get_or_update_translations(model, tokenizer, test_data, TRAIN_DIRECTION, update=UPDATE_ALL_TEST)

### 9.3. Tính toán và Báo cáo BLEU

Cell này tải kết quả từ Hub và tính **SacreBLEU** cho cả hai chiều:

#### Output mẫu:
```
| Direction |   BASE   | FINE-TUNED | Improvement|
|-----------|----------|------------|------------|
| EN -> VI  |   45.23  |      52.67 |      +7.44 |
| VI -> EN  |   48.91  |      55.12 |      +6.21 |
```

**Giải thích:**
- Cải thiện đáng kể (~6-7 điểm BLEU) chứng minh hiệu quả của LoRA fine-tuning
- Điểm VI→EN thường cao hơn do tiếng Anh y khoa có tính khuôn mẫu cao

In [ ]:
def calculate_bleu_from_df(df):
    if df is None: return 0.0, 0.0
    refs = [df["reference"].tolist()]
    b_base = sacrebleu.corpus_bleu(df["base_model"].tolist(), refs).score
    b_ft = sacrebleu.corpus_bleu(df["fine_tuned"].tolist(), refs).score
    return b_base, b_ft

def full_report():
    print("\n" + "="*50)
    print("BÁO CÁO SO SÁNH HIỆU NĂNG (DỰA TRÊN DỮ LIỆU ĐÃ LƯU)")
    print("="*50)

    # Load file EN->VI
    try:
        path_en_vi = hf_hub_download(repo_id=TEST_REPO_ID, filename=FILE_EN_VI, repo_type="dataset")
        df_en_vi = pd.read_parquet(path_en_vi)
        base_en_vi, ft_en_vi = calculate_bleu_from_df(df_en_vi)
    except:
        base_en_vi, ft_en_vi = 0, 0

    # Load file VI->EN
    try:
        path_vi_en = hf_hub_download(repo_id=TEST_REPO_ID, filename=FILE_VI_EN, repo_type="dataset")
        df_vi_en = pd.read_parquet(path_vi_en)
        base_vi_en, ft_vi_en = calculate_bleu_from_df(df_vi_en)
    except:
        base_vi_en, ft_vi_en = 0, 0

    # In bảng kết quả giống hệt format cũ của bạn
    print(f"| Direction |   BASE   | FINE-TUNED | Improvement|")
    print("|-----------|----------|------------|------------|")
    
    imp_en_vi = ft_en_vi - base_en_vi
    print(f"| EN -> VI  | {base_en_vi:8.2f} | {ft_en_vi:10.2f} | {imp_en_vi:10.2f} |")
    
    imp_vi_en = ft_vi_en - base_vi_en
    print(f"| VI -> EN  | {base_vi_en:8.2f} | {ft_vi_en:10.2f} | {imp_vi_en:10.2f} |")
    print("="*50)

full_report()

## 10. Đánh giá Định tính (Gemini Score)

### Cài đặt Google Gemini API
Nâng cấp thư viện `google-genai` để sử dụng Gemini làm LLM-as-a-Judge.

In [ ]:
!pip install -q -U google-genai sacrebleu

### 10.1. Chuẩn bị API Keys

Sử dụng **8 API keys** để xoay vòng khi gặp giới hạn rate limit (429 error).

In [ ]:
import json
import time
import random
import torch
from google import genai
from tqdm import tqdm
import sacrebleu

test_data = raw_dataset["test"]

API_KEYS = ["API KEY"
]

### 10.2. Hàm Chấm điểm Song song (Dual Scoring)

#### Tiêu chí Đánh giá (0-100 điểm):

1. **Độ chính xác y khoa (50%)**:
   - Sai thuật ngữ/liều lượng: -40 điểm
   - Bỏ sót thông tin quan trọng: -20 điểm

2. **Thuật ngữ chuyên ngành (30%)**:
   - Ưu tiên từ chuyên môn hơn từ phổ thông

3. **Văn phong & Tính lưu loát (20%)**:
   - Văn phong trung lập, khoa học
   - Không chứa câu dẫn ("Here is the translation...")

#### Cơ chế Fallback:
- Ưu tiên `gemini-2.0-flash`
- Nếu 404 → Chuyển sang `gemini-2.0-flash-lite`
- Nếu 429 (hết quota) → Đổi API key và retry

In [ ]:
def get_dual_gemini_scores(batch_items, api_keys, api_idx, src_lang, tgt_lang):
    """
    Chấm điểm song song Base vs FT với cơ chế đổi Key và Fallback Model.
    """
    # Lấy Key hiện tại dựa trên api_idx
    current_key = api_keys[api_idx % len(api_keys)]
    client = genai.Client(api_key=current_key)
    
    num_items = len(batch_items)
    items_prompt = ""
    for idx, item in enumerate(batch_items):
        items_prompt += f"""
--- Cặp {idx+1} ---
Gốc: {item['src']}
Tham chiếu: {item['ref']}
Bản dịch A (Base): {item['base']}
Bản dịch B (Fine-tuned): {item['ft']}
"""

    prompt = f"""
        Bạn là một chuyên gia thẩm định dịch thuật Y khoa (Medical Translation Evaluator).
    
        Nhiệm vụ của bạn là đánh giá hai bản dịch từ {src_lang} sang {tgt_lang}:
        - Bản dịch A: Base model
        - Bản dịch B: Fine-tuned model
        
        Việc đánh giá phải dựa trên câu tham chiếu và tập trung vào tính an toàn y khoa.
        
        ### TIÊU CHÍ CHẤM ĐIỂM (0–100):
        1. Độ chính xác y khoa (50%):
           - Sai thuật ngữ bệnh, thuốc, liều lượng hoặc chỉ dẫn lâm sàng → trừ ≥40 điểm.
           - Bỏ sót thông tin y khoa quan trọng (omission) → trừ ≥20 điểm.
        
        2. Thuật ngữ chuyên ngành (30%):
           - Ưu tiên thuật ngữ y khoa chuẩn, nhất quán.
           - Dùng từ phổ thông thay cho thuật ngữ chuyên môn → trừ điểm.
        
        3. Văn phong & tính lưu loát (20%):
           - Văn phong trung lập, khoa học.
           - Không chứa câu dẫn như "Here is the translation".
        
        ### QUY TẮC TRỪ ĐIỂM CỨNG:
        - Trừ 10 điểm: Có câu dẫn hoặc bình luận ngoài nội dung dịch.
        - Trừ 40 điểm: Lỗi nghiêm trọng ảnh hưởng an toàn y tế.
        - Trừ 20 điểm: Cấu trúc danh từ y khoa gây mơ hồ nghĩa.
        
        ### ĐỊNH DẠNG TRẢ VỀ (CHỈ JSON):
        {{
          "results": [
            {{
              "base_score": int,
              "ft_score": int,
              "reason": "Nhận xét ngắn gọn, tập trung vào lỗi hoặc ưu điểm y khoa"
            }}
          ]
        }}
        
        Dữ liệu cần chấm điểm:
    {items_prompt}"""

    # Danh sách model ứng viên theo thứ tự ưu tiên
    model_candidates = [
        "models/gemini-2.0-flash", 
        "models/gemini-2.0-flash-lite"
    ]

    for model_id in model_candidates:
        try:
            response = client.models.generate_content(
                model=model_id,
                contents=prompt
            )
            text = response.text.strip()
            
            # Làm sạch JSON từ markdown
            if "```json" in text:
                text = text.split("```json")[1].split("```")[0].strip()
            elif "```" in text:
                text = text.replace("```", "").strip()
            
            data = json.loads(text)
            results = data.get("results", [])
            
            if len(results) == num_items:
                return results, api_idx + 1
                
        except Exception as e:
            if "404" in str(e):
                continue # Thử model tiếp theo
            elif "429" in str(e):
                # Hết quota -> Đổi Key ngay lập tức và đệ quy lại với chính batch này
                print(f"Key {api_idx % len(api_keys)} hết hạn mức. Đang đổi sang Key tiếp theo...")
                return get_dual_gemini_scores(batch_items, api_keys, api_idx + 1, src_lang, tgt_lang)
            else:
                print(f"Lỗi không xác định: {e}")
                break
                
    return [{"base_score": None, "ft_score": None, "reason": "Error"}] * num_items, api_idx + 1

### 10.3. Thực thi Đánh giá Gemini

#### Quy trình:
1. **Lọc**: Chọn 50 câu có độ dài ≥15 từ (câu phức tạp, giàu thuật ngữ)
2. **Batch Scoring**: Chấm điểm theo lô 4 mẫu/lần
3. **Lưu trữ**: Lưu DataFrame đầy đủ (gồm source, reference, base, ft, scores, reason) lên Hub

#### Output:
- `base_score`: Điểm Base Model
- `ft_score`: Điểm Fine-tuned Model
- `reason`: Nhận xét chi tiết của Gemini về lỗi/ưu điểm

**Kết quả mẫu:**
```
✅ Đánh giá xong! Trung bình Base: 72.35 | FT: 85.12
```

In [ ]:
def run_gemini_comparison_eval(num_samples=100, min_words=15):
    # 1. Load dữ liệu đã dịch sẵn từ Hub
    filename = FILE_EN_VI if TRAIN_DIRECTION == "en_vi" else FILE_VI_EN
    try:
        path = hf_hub_download(repo_id=TEST_REPO_ID, filename=filename, repo_type="dataset")
        df = pd.read_parquet(path)
    except Exception as e:
        print(f"Không tìm thấy file dữ liệu: {e}")
        return None

    # Lọc các câu có độ dài >= min_words (tính theo số từ)
    # Điều này giúp đánh giá các câu phức hợp, mang tính chuyên môn cao hơn
    df_filtered = df[df['source'].str.split().str.len() >= min_words].copy()
    
    if len(df_filtered) < num_samples:
        print(f"Chỉ tìm thấy {len(df_filtered)} câu thỏa mãn độ dài >= {min_words}. Sẽ dùng toàn bộ.")
        num_samples = len(df_filtered)

    # Lấy mẫu ngẫu nhiên từ danh sách đã lọc
    sampled_df = df_filtered.sample(n=num_samples, random_state=42).copy()
    
    src_lang = "Tiếng Anh" if TRAIN_DIRECTION == "en_vi" else "Tiếng Việt"
    tgt_lang = "Tiếng Việt" if TRAIN_DIRECTION == "en_vi" else "Tiếng Anh"
    
    items_to_score = []
    for _, row in sampled_df.iterrows():
        items_to_score.append({
            "src": row["source"], "ref": row["reference"], 
            "base": row["base_model"], "ft": row["fine_tuned"]
        })

    all_results = []
    api_idx = 0
    batch_size = 4 

    print(f"Đang chấm điểm {len(items_to_score)} câu (Độ dài >= {min_words} từ)...")
    for i in tqdm(range(0, len(items_to_score), batch_size), desc="Gemini Comparison"):
        batch = items_to_score[i:i+batch_size]
        results, api_idx = get_dual_gemini_scores(batch, API_KEYS, api_idx, src_lang, tgt_lang)
        all_results.extend(results)
        time.sleep(4)

    # Tổng hợp tất cả thông tin vào DataFrame
    # Cột đã có: source, reference, base_model, fine_tuned
    # Cột thêm mới:
    sampled_df["base_score"] = [r.get("base_score") for r in all_results]
    sampled_df["ft_score"] = [r.get("ft_score") for r in all_results]
    sampled_df["reason"] = [r.get("reason") for r in all_results]

    # In kết quả tổng quát
    avg_base = sampled_df["base_score"].dropna().mean()
    avg_ft = sampled_df["ft_score"].dropna().mean()
    print(f"\nĐánh giá xong! Trung bình Base: {avg_base:.2f} | FT: {avg_ft:.2f}")

    # Lưu file eval chi tiết (chứa mọi cột) lên Hub
    eval_file = f"gemini_dual_eval_{TRAIN_DIRECTION}.parquet"
    sampled_df.to_parquet(eval_file)
    upload_file(path_or_fileobj=eval_file, path_in_repo=eval_file, repo_id=TEST_REPO_ID, repo_type="dataset")
    
    return sampled_df

# Thực thi
df_comparison = run_gemini_comparison_eval(num_samples=50, min_words=15)

## 11. Demo Dịch Thuật Trực tiếp

### So sánh Base Model vs Fine-tuned Model

Cell này cho phép kiểm thử nhanh với một câu cụ thể:

#### Ví dụ:
**Input (VI):**
> "Nghiên cứu được thực hiện nhằm đánh giá tác dụng giảm cân, hạ lipid máu của hỗn hợp dịch chiết lá Trà hoa vàng và Giảo cổ lam trên chuột nhắt trắng gây béo phì."

**Output (EN):**
- **Base Model**: Bản dịch chưa tối ưu, có thể thiếu thuật ngữ chính xác
- **Fine-tuned Model**: Bản dịch chính xác, sử dụng thuật ngữ khoa học chuẩn

#### Tham số:
- `target_lang="en"`: Dịch sang tiếng Anh
- `target_lang="vi"`: Dịch sang tiếng Việt
- `temperature=0.3`: Giảm randomness, tăng tính nhất quán
- `repetition_penalty=1.1`: Tránh lặp từ

In [ ]:

import torch
# dịch 1 ví dụ
def translate_wrapper(text, model, tokenizer, label="", target_lang="en"):
    """
    target_lang="vi" -> Dịch sang Tiếng Việt
    target_lang="en" -> Dịch sang Tiếng Anh
    """
    
    if target_lang == "en":
        prompt = f"Dịch câu sau sang tiếng Anh:\n{text}"
    else:
        prompt = f"Dịch câu sau sang tiếng Việt:\n{text}"
        
    messages = [{"role": "user", "content": prompt}]
    
    inputs = tokenizer.apply_chat_template(
        messages, tokenize=True, add_generation_prompt=True, return_tensors="pt"
    ).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=512,
            temperature=0.3,
            do_sample=True,
            top_p=0.9,
            repetition_penalty=1.1
        )
    
    generated_ids = outputs[0][len(inputs[0]):]
    result = tokenizer.decode(generated_ids, skip_special_tokens=True)
    
    print(f"[{label}]: {result}")
    return result

def compare_models(text_input, model, tokenizer, target_lang="en"):
    direction = "VIỆT -> ANH" if target_lang == "en" else "ANH -> VIỆT"
    print(f"Input ({direction}): {text_input}\n" + "-"*50)
    
    print(">>> 1. Model Gốc (Base Model):")
    with model.disable_adapter():
        old_pred = translate_wrapper(text_input, model, tokenizer, label="Old", target_lang=target_lang)
        
    print("-" * 30)
    
    print(">>> 2. Model Mới (Fine-tuned):")
    new_pred = translate_wrapper(text_input, model, tokenizer, label="New", target_lang=target_lang)
    
    print("=" * 50 + "\n")

sample_vi = "Nghiên cứu được thực hiện nhằm đánh giá tác dụng giảm cân, hạ lipid máu của hỗn hợp dịch chiết lá Trà hoa vàng và Giảo cổ lam trên chuột nhắt trắng gây béo phì."
sample_en = "The purpose of this study was to evaluate the effects of a mixture extract of C chrysantha and G pentaphyllum on weight loss and lowering lipid blood levels in obese Swiss mice."


print(f"Reference English: {sample_en}\n")

# Gọi hàm với tham số target_lang="en"
compare_models(sample_vi, model, tokenizer, target_lang="en")
# Gọi hàm với tham số target_lang="vi"
compare_models(sample_en, model, tokenizer, target_lang="vi")